## Making the red SPIRE catalogue

![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>)


### The method we used is decsribed in Asboth et al. 2016.
### Our method creates a 6 arcsec 500um map by using bicubic interpolation.
### The D-map is created in the same way as Asboth et al. 2016, and we select all sources with a 3sigma detection in the D-map. We extract the flux density of the sources by using the same method as used for the standart blind cathalogue. 

#### The RED SPIRE sources are validated with the xxxx notebook

In [3]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
import os

import func_make_cat as fc
import matplotlib.pyplot as plt
from astropy.wcs import WCS
from matplotlib.patches import Ellipse
from astropy.coordinates import SkyCoord
from astropy import units as u
%matplotlib inline  
from reproject import reproject_interp
from reproject import reproject_exact
import func_make_cat as fc


loc = "data_HELP_v1.0/"


In [52]:
all_names =  ['GAMA-09_SPIRE','GAMA-12_SPIRE','GAMA-15_SPIRE','HATLAS-NGP_SPIRE','HATLAS-SGP_SPIRE','SSDF_SPIRE',\
              'AKARI-SEP_SPIRE','Bootes_SPIRE','CDFS-SWIRE_SPIRE','COSMOS_SPIRE','EGS_SPIRE',\
              'ELAIS-N1_SPIRE','ELAIS-N2_SPIRE','ELAIS-S1_SPIRE','HDF-N_SPIRE','Lockman-SWIRE_SPIRE','SA13_SPIRE',\
              'SPIRE-NEP_SPIRE','xFLS_SPIRE','XMM-13hr_SPIRE','XMM-LSS_SPIRE','AKARI-NEP_SPIRE']
band = ['250','350','500']
        

In [50]:
#reload(fc)
for j in range(np.size(all_names)):    
    print(all_names[j]+band[0]+'_v1.0.fits')
    name = all_names[j]+band[0]+'_v1.0.fits'
    hdulist250 = fits.open(loc+name)
    
    name = all_names[j]+band[1]+'_v1.0.fits'
    hdulist350 = fits.open(loc+name)

    name = all_names[j]+band[2]+'_v1.0.fits'
    hdulist500 = fits.open(loc+name)

    # https://reproject.readthedocs.io/en/stable/ for info about the reproject function
    # order 1 pics the nearest pixel from the projected header
    new_image0, footprint0 = reproject_interp(hdulist500["MFILT"], hdulist250['IMAGE'].header, order = 0) 

    # NaN pixels in the orignal 500um 
    # These locations are needed as the Bicubic interpolation needs NaN's to be set to 0
    bad = np.isnan(hdulist500["MFILT"].data)
    hdulist500["MFILT"].data[bad] = 0 

    new_image2, footprint2 = reproject_interp(hdulist500["MFILT"], hdulist250['IMAGE'].header, order = 2) 
    bad = np.isnan(new_image0)
    new_image2[bad] = np.nan

    new_D = fc.do_filtering(hdulist250,hdulist500,new_image2)
    
    new_image2 -= np.nanmean(new_image2) # make sure the new image is mean substracted
    Dhdu = fits.ImageHDU(header=hdulist250['IMAGE'].header,data=new_image2)
    Dhdu.writeto('Red_map/'+all_names[j]+'_map_500_6ac.fits') 

    Dhdu = fits.ImageHDU(header=hdulist250['IMAGE'].header,data=new_D)
    Dhdu.writeto('Red_map/'+all_names[j]+'_D_map.fits') 
    Dhdu = fits.open('Red_map/'+all_names[j]+'_D_map.fits')

    dmin = 3*np.nanstd(Dhdu[1].data)
    dp, rap, decp, xp, yp = fc.find_peak_red(Dhdu,dmin)
    

    
    if all_names[j] == 'AKARI-NEP_SPIRE':
        import pyregion
        x_all = np.arange(0,np.shape(Dhdu[1].data)[0])
        y_all = np.arange(0,np.shape(Dhdu[1].data)[1])
        x_mat = np.tile(x_all,np.size(y_all))
        y_mat = np.repeat(y_all,np.size(x_all))
    
        region_name = "AKARI-NEP.reg"
        r = pyregion.open(region_name)
        r250 = r.get_filter(hdulist250[1].header)
        
        mask = r250.inside(y_mat,x_mat)
        Dhdu[1].data[x_mat[~mask],y_mat[~mask]] = np.nan
        Dhdu.writeto('Red_map/'+all_names[j]+'_D_map3.fits') 

        dmin = 3*np.nanstd(Dhdu[1].data)
        dp, rap, decp, xp, yp = np.array(fc.find_peak_red(Dhdu,dmin))        



    w1 = WCS(hdulist250["NEBFILT"].header) 
    w2 = WCS(hdulist350["NEBFILT"].header)
    w3 = WCS(hdulist500["MFILT"].header)    
    
    x_250, y_250 = np.round(w1.wcs_world2pix(rap,  decp, 0)).astype(int)
    x_350, y_350 = np.round(w2.wcs_world2pix(rap,  decp, 0)).astype(int)
    x_500, y_500 = np.round(w3.wcs_world2pix(rap,  decp, 0)).astype(int)
    
    S250 = hdulist250["NEBFILT"].data[y_250,x_250]
    E250 = hdulist250["ERROR"].data[y_250,x_250]
    S350 = hdulist350["NEBFILT"].data[y_350,x_350]
    E350 = hdulist350["ERROR"].data[y_350,x_350]
    S500 = hdulist500["NEBFILT"].data[y_500,x_500]
    E500 = hdulist500["ERROR"].data[y_500,x_500]

    hdu = fits.BinTableHDU.from_columns(\
         [fits.Column(name='RA', array=rap, format ='F'),
         fits.Column(name='Dec', array=decp, format='F'),
         fits.Column(name='F_RED_pix_SPIRE_250', array=S250, format ='F'),
         fits.Column(name='FErr_RED_pix_SPIRE_250', array=E250, format ='F'),          
         fits.Column(name='F_RED_pix_SPIRE_350', array=S350, format ='F'),
         fits.Column(name='FErr_RED_pix_SPIRE_350', array=E350, format ='F'),
         fits.Column(name='F_RED_pix_SPIRE_500', array=S500, format ='F'),
         fits.Column(name='FErr_RED_pix_SPIRE_500', array=E500, format ='F')         
         ])
    hdu.writeto('Red_map/'+all_names[j]+'_D_cat.fits')
    
    
    hdulist250.close()
    hdulist350.close()
    hdulist500.close()


AKARI-NEP_SPIRE250_v1.0.fits


In [54]:
for j in range(np.size(all_names)):   
    print(loc+all_names[j]+band[0]+'_v1.0.fits')

    name = all_names[j]+band[0]+'_v1.0.fits'
    hdulist1 = fits.open(loc+name)
    name = all_names[j]+band[1]+'_v1.0.fits'
    hdulist2 = fits.open(loc+name)
    name = all_names[j]+band[2]+'_v1.0.fits'
    hdulist3 = fits.open(loc+name)

    ff = fits.open('Red_map/'+all_names[j]+'_D_cat2.fits')
    ra,dec = (ff[1].data)['RA'],(ff[1].data)['Dec']

    r_1, ra_1, dec_1, S250_1, E250_1, S350_1, E350_1, S500_1, E500_1 = \
                fc.corr_psf_max_MF(hdulist1,hdulist2,hdulist3,ra, dec)
    hdu = fits.BinTableHDU.from_columns(\
         [fits.Column(name='RA', array=ra_1, format ='F',unit='degree'),
         fits.Column(name='Dec', array=dec_1, format='F',unit='degree'),
         fits.Column(name='F_RED_MF_SPIRE_250', array=1000*S250_1, format ='F',unit='mJy'),
         fits.Column(name='FErr_RED_MF_SPIRE_250', array=1000*E250_1, format ='F',unit='mJy'),
         fits.Column(name='F_RED_MF_SPIRE_350', array=1000*S350_1, format ='F',unit='mJy'),
         fits.Column(name='FErr_RED_MF_SPIRE_350', array=1000*E350_1, format ='F',unit='mJy'),
         fits.Column(name='F_RED_MF_SPIRE_500', array=1000*S500_1, format ='F',unit='mJy'),
         fits.Column(name='FErr_RED_MF_SPIRE_500', array=1000*E500_1, format ='F',unit='mJy'),
         fits.Column(name='r', array=r_1, format ='F'),
         fits.Column(name='RA_pix', array=(ff[1].data)['RA'], format ='F'), 
         fits.Column(name='Dec_pix', array=(ff[1].data)['Dec'], format ='F'), 
         fits.Column(name='F_RED_pix_SPIRE_250', array=1000*(ff[1].data)['F_RED_pix_SPIRE_250'], format ='F',unit='mJy'), 
         fits.Column(name='FErr_RED_pix_SPIRE_250', array=1000*(ff[1].data)['FErr_RED_pix_SPIRE_250'], format ='F',unit='mJy'), 
         fits.Column(name='F_RED_pix_SPIRE_350', array=1000*(ff[1].data)['F_RED_pix_SPIRE_350'], format ='F',unit='mJy'), 
         fits.Column(name='FErr_RED_pix_SPIRE_350', array=1000*(ff[1].data)['FErr_RED_pix_SPIRE_350'], format ='F',unit='mJy'), 
         fits.Column(name='F_RED_pix_SPIRE_500', array=1000*(ff[1].data)['F_RED_pix_SPIRE_500'], format ='F',unit='mJy'), 
         fits.Column(name='FErr_RED_pix_SPIRE_500', array=1000*(ff[1].data)['FErr_RED_pix_SPIRE_500'], format ='F',unit='mJy') 
         ])
    hdu.writeto('Red_map/'+all_names[j]+'_D_MF_cat.fits') 

data_HELP_v1.0/SPIRE-NEP_SPIRE250_v1.0.fits
(0.0, '%')
(4.0, '%')
(8.0, '%')
(12.0, '%')
(16.0, '%')
(20.0, '%')
(24.0, '%')
(28.000000000000004, '%')
(32.0, '%')
(36.0, '%')
(40.0, '%')
(44.0, '%')
(48.0, '%')
(52.0, '%')
(56.00000000000001, '%')
(60.0, '%')
(64.0, '%')
(68.0, '%')
(72.0, '%')
(76.0, '%')
(80.0, '%')
(84.0, '%')
(88.0, '%')
(92.0, '%')
(96.0, '%')


![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**: Steven Duivenvoorden 

For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).

The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)